In [4]:
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd /content/drive/MyDrive/4Semestre/clase_consultoria/ramon

/content/drive/MyDrive/4Semestre/clase_consultoria/ramon


In [7]:
# load data
ventas = pd.read_csv("data/ventas_sku_loc_week.csv")


# Ordenamos los datos por semana

In [8]:
fechas = ventas.FECHA.unique()

semanas = {fechas[i]:'w'+str(i+1) for i in range(len(fechas))} #key(w*), value(yyyy-mm-dd)

week_df = ventas[ventas.FECHA == fechas[0]]
week_df = week_df.loc[:,["SKU", "LOC"]]
week_df = week_df.reset_index()

for i in range(len(fechas)):
  pass
  ventas_sem = ventas[ventas.FECHA == fechas[i]]
  ventas_sem = ventas_sem.reset_index()
  unidades_sem = ventas_sem['UNI']
  
  week_df[semanas[fechas[i]]] = unidades_sem
#  print(ventas_sem.shape)
week_df = week_df.iloc[:,1:]
week_df.to_csv("data/weekly_data.csv")
week_df.head()

,SKU,LOC,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12
0,Art01,LOC_001,2.0,8.0,2.0,10.0,3.0,14.0,11.0,17.0,9.0,13.0,8.0,6.0
1,Art01,LOC_002,9.0,4.0,7.0,6.0,6.0,1.0,3.0,3.0,3.0,8.0,3.0,2.0
2,Art01,LOC_003,3.0,8.0,8.0,5.0,6.0,5.0,7.0,4.0,4.0,6.0,9.0,12.0
3,Art01,LOC_004,9.0,14.0,4.0,12.0,4.0,15.0,14.0,12.0,9.0,18.0,8.0,12.0
4,Art01,LOC_005,12.0,16.0,15.0,11.0,3.0,6.0,9.0,12.0,9.0,6.0,13.0,5.0


In [ ]:
# last two weeks for SKU and LOC 
ventas_last_two_weeks = ventas.groupby(["SKU", "LOC"], as_index=False).tail(2)

# transform table
ventas_last_two_weeks_pivot = ventas_last_two_weeks.pivot_table(columns="FECHA", 
                                                                values="UNI", index=["SKU", "LOC"]).reset_index()

# rename columns
ventas_last_two_weeks_pivot.rename(columns={"2019-06-16":"penultimate_week",
                                           "2019-06-23":"last_week"}, inplace=True)

# create forecasts baseline
for i in range(4):
    ventas_last_two_weeks_pivot["baseline_predict_week"+str(i)] = ventas_last_two_weeks_pivot[["penultimate_week", "last_week"]].apply(lambda x: x.mean()+1, axis=1)

In [ ]:
ventas_last_two_weeks_pivot.to_csv("../data/ventas_forecast_baseline.csv", index=False)